In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("C:\\Users\\MD USMAN ANSARI\\BO\\Top Indian Places to Visit.csv")

In [4]:
# Analyze the dataset structure to understand available columns and data types
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325 entries, 0 to 324
Data columns (total 16 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Unnamed: 0                        325 non-null    int64  
 1   Zone                              325 non-null    object 
 2   State                             325 non-null    object 
 3   City                              325 non-null    object 
 4   Name                              325 non-null    object 
 5   Type                              325 non-null    object 
 6   Establishment Year                325 non-null    object 
 7   time needed to visit in hrs       325 non-null    float64
 8   Google review rating              325 non-null    float64
 9   Entrance Fee in INR               325 non-null    int64  
 10  Airport with 50km Radius          325 non-null    object 
 11  Weekly Off                        32 non-null     object 
 12  Signific

,Unnamed: 0,Zone,State,City,Name,Type,Establishment Year,time needed to visit in hrs,Google review rating,Entrance Fee in INR,Airport with 50km Radius,Weekly Off,Significance,DSLR Allowed,Number of google review in lakhs,Best Time to visit
0,0,Northern,Delhi,Delhi,India Gate,War Memorial,1921,0.5,4.6,0,Yes,NaN,Historical,Yes,2.60,Evening
1,1,Northern,Delhi,Delhi,Humayun's Tomb,Tomb,1572,2.0,4.5,30,Yes,NaN,Historical,Yes,0.40,Afternoon
2,2,Northern,Delhi,Delhi,Akshardham Temple,Temple,2005,5.0,4.6,60,Yes,NaN,Religious,No,0.40,Afternoon
3,3,Northern,Delhi,Delhi,Waste to Wonder Park,Theme Park,2019,2.0,4.1,50,Yes,Monday,Environmental,Yes,0.27,Evening
4,4,Northern,Delhi,Delhi,Jantar Mantar,Observatory,1724,2.0,4.2,15,Yes,NaN,Scientific,Yes,0.31,Morning


### Recommendation Model Design
We will create a function that takes a 'Source City' as input and recommends the top 5 weekend destinations. The ranking will be based on:
- **Distance** 
- **Google review rating**
- **Number of Google reviews (popularity)**

In [7]:

def recommend_weekend_destinations(source_city, df, top_n=3):
    # Find the zone and state of the source city
    source_info = df[df['City'].str.lower() == source_city.lower()]
    if source_info.empty:
        return f"Source city '{source_city}' not found in dataset."
    
    source_zone = source_info.iloc[0]['Zone']
    source_state = source_info.iloc[0]['State']
    
    # Filter destinations in the same zone or state, but not the source city itself
    candidates = df[((df['Zone'] == source_zone) | (df['State'] == source_state)) & (df['City'].str.lower() != source_city.lower())]
    
    # Score: weighted sum of rating and popularity
    # Normalize rating and number of reviews
    candidates = candidates.copy()
    candidates['rating_norm'] = candidates['Google review rating'] / candidates['Google review rating'].max()
    candidates['popularity_norm'] = candidates['Number of google review in lakhs'] / candidates['Number of google review in lakhs'].max()
    
    # Simple scoring: 60% rating, 40% popularity
    candidates['score'] = 0.6 * candidates['rating_norm'] + 0.4 * candidates['popularity_norm']
    
    # Sort by score descending
    top_destinations = candidates.sort_values('score', ascending=False).head(top_n)
    
    return top_destinations[['Name', 'City', 'State', 'Zone', 'Google review rating', 'Number of google review in lakhs', 'score']]

In [15]:
city = input("Enter your Sourse City: ")
print(f"Top 3 Recomended Destinations from city '{city}' are--")
recommend_weekend_destinations(city, df)

Top 3 Recomended Destinations from city 'goa' are--


,Name,City,State,Zone,Google review rating,Number of google review in lakhs,score
111,Mysore Palace,Mysore,Karnataka,Southern,4.6,2.5,0.9750
101,Wayanad Wildlife Sanctuary,Wayanad,Kerala,Southern,4.5,2.2,0.9145
32,Charminar,Hyderabad,Telangana,Southern,4.5,2.1,0.8985
